In [ ]:
import pandas as pd
import torch
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
import wandb

from transformers import AlbertTokenizer, TFAlbertModel
import numpy as np
import itertools

In [ ]:
# Data pre-processing and data loading functions

"""
Loading dataset from excel

Return:
    Data file read from Pandas    
"""

train = pd.read_excel('../dataset/Project_Datasets/Constraint_English_Train.xlsx')
val = pd.read_excel('../dataset/Project_Datasets/Constraint_English_Val.xlsx')
    

"""
Clean Text function

Args:
    string: Each line of tweets from the excel file
    
Output:
    The processed tweets according to cleanText criteria
"""

stops = set(stopwords.words("english"))
def cleanText(string):
    text = string.lower().split() # change all the sentences to lower case
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text) # removes link    
    text = re.sub(r"www(\S)+",' ',text) # removes link
    text = text.replace('&amp',' ') # remove &amp and replace with space
    text = re.sub(r"&",' and ',text) # replace the symbol "&" with the word "and"
    text = re.sub(r"[^0-9a-zA-Z]+",' ',text) # removes all except regex (including emojis)
    text = text.split()
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text


"""
Data loader function

Args:
    input_data: Dataset excel file read from Pandas
    
Output: 
    List of lists, each list is of [tweets, label]
"""  

def load_data(input_data):
    output = []
    for index, row in input_data.iterrows():
        tweets = cleanText(row[1])
        label = row[2]
        if label == "real":
            label_out = 1
        else:
            label_out = 0
        output.append([tweets, label_out])
    return output


print(load_data(train))

In [ ]:
# Saving post-processed Training output

"""
This code will save the post processed training dataset into a text file

Output:
    .txt file with each line as an input from the excel in the format of [tweet, label]
"""

post_processed = load_data(train)

with open('post_processed_output.txt', 'w') as f:
    for item in post_processed:
        f.write("%s - %d\n" %(item[0], item[1]))

In [ ]:
# Loading Train, Val data in Pandas.DataFrame format

train_data = load_data(train)
train_df = pd.DataFrame(train_data)
train_df.columns = ["tweets", "labels"]

val_data = load_data(val)
val_df = pd.DataFrame(val_data)
val_df.columns = ["tweets", "labels"]

In [ ]:
# For hyperparameter tuning
# This cell should be skipped if the intention is not for tuning hyperparameters

from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args.eval_batch_size = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = 1000
model_args.learning_rate = 4e-4
model_args.manual_seed = 4
model_args.max_seq_length = 256
model_args.multiprocessing_chunksize = 5000
model_args.no_cache = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 16
model_args.gradient_accumulation_steps = 2
model_args.train_custom_parameters_only = False
model_args.wandb_project = "albert"

"""
For weights & bias (wandb) configuration and running

Required:
    sweep_config: Configuration for the hyperparameters to be tuned
    wandb.login(): For connecting to the wandb website with the credentials
    sweep_id: configure which project the sweep is running for
"""

sweep_config = {
    "name": "batch-size-test",
    "method": "bayes",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"min": 30, "max": 60},
        "learning_rate": {"min": 0, "max": 1e-3},
        "max_seq_length": {"values": [512, 256, 128, 64]},
        "batch_size": {"values": [256, 128, 64]}
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,},
}

wandb.login()

sweep_id = wandb.sweep(sweep_config, project="albert")

"""
This function is used by wandb agent for sweep hyperparameter tuning

Parameters:
    Model: specify which model to use (eg. albert, bert)
    Version: specify which version of the model (eg. albert-base-v2)
    wandb.config: specify the config (can be from a external config file
                                      or the sweep_config declared earlier on)
"""
def train():

    # Initialize a new wandb run
    wandb.init() 
    
    model = ClassificationModel(
        'albert', 
        'albert-base-v2', 
        num_labels=2, 
        use_cuda=True, 
        args=model_args,
        sweep_config=wandb.config
    )
    
    model.train_model(
        train_df,
        eval_df=val_df,
        accuracy=lambda truth, predictions: accuracy_score(
            truth, [round(p) for p in predictions]
        )
    )

    # Sync wandb
    wandb.join()

"""
A worker agent that takes in sweep_id and train() function
to start the sweep and to be monitored in wandb website for results
"""
wandb.agent(sweep_id, train)

In [ ]:
# For training the Model

from simpletransformers.classification import ClassificationModel, ClassificationArgs

"""
The following set of hyperparameters are the optimized hyperparameters
"""
model_args = ClassificationArgs()
model_args.train_batch_size = 64
model_args.learning_rate = 0.0000215
model_args.max_seq_length = 256
model_args.num_train_epochs = 42
model_args.overwrite_output_dir = True

model = ClassificationModel(
        'albert', 
        'albert-base-v2', 
        num_labels=2, 
        use_cuda=True, 
        args=model_args
    )

# Training the model
step, tr_loss = model.train_model(train_df)

# Evaluation during training
result, model_outputs, wrong_preds = model.eval_model(val_df)

In [ ]:
# Evaluating the Model

# Loading the test dataset
test = pd.read_excel('../dataset/Project_Datasets/english_test_with_labels.xlsx')

"""
Predict function 

Input:
    input_data: The test dataset to be used for evaluation
    
Output:
    List of lists with each line in the form of [tweet, ground_truth, prediction]
        for line in the test dataset excel
"""
def predict(input_data):
    output = []
    for index, row in input_data.iterrows():        
        tweets = cleanText(row[1])
        gt_label= row[2]
        predictions, raw_outputs = model.predict([tweets])
        output.append([tweets, gt_label , predictions])
    return output

pred_output = predict(test)

print(pred_output)

In [ ]:
# Save the prediction output to a file

"""
This code will save the prediction output into a text file

Output:
    .txt file with each line as an ouput from evaluating the model 
        in the format of [tweet, ground_truth, prediction]
"""

with open('pred_output.txt', 'w') as f:
    for item in pred_output:
        f.write("%s - %s - %d\n" %(item[0], item[1], item[2]))

In [ ]:
# Evaluation metric calculation

"""
This function will calculate the various evaluation metrics

Input:
    lists: list of lists that is output from the predict() function

Output:
    TP: True positive
    TN: True negative
    FP: False positive
    FN: False negative
    accuracy: calculated as (TP + TN) / (TP + FP + FN + TN)
    precision: calculated as TP / (TP + FP)
    recall: calculated as TP / (TP + FN)
"""

def eval_metric_calc(lists):
    precision = 0
    recall = 0
    accuracy = 0
    TP = 0 
    FP = 0
    FN = 0
    TN = 0
    
    for i in range(len(lists)):
        if lists[i][1] == "fake" and lists[i][2] == 0:
            TP += 1
        elif lists[i][1] == "fake" and lists[i][2] == 1:
            FN += 1
        elif lists[i][1] == "real" and lists[i][2] == 0:
            FP += 1
        else:
            TN += 1
            
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    
    return TP, TN, FP, FN, accuracy, precision, recall

TP, TN, FP, FN, accuracy, precision, recall = eval_metric_calc(pred_output)

print("accuracy : ", accuracy, "precision : ", precision, "recall : ", recall)
print("TP : ", TP, "TN : ", TN, "FP : ", FP, "FN : ", FN)

In [ ]:
# Visualization of Results

labels = ['True Positive', 'True Negative']
ALBERT = [TP, TN] # Metric from Model evaluation
SVM = [1039, 958] # Hardcoded value obtained from supervised baseline

x = np.arange(len(labels))  # The label locations
width = 0.15  # The width of the bars

fig, ax = plt.subplots()
albert = ax.bar(x - width/2, ALBERT, width, label='ALBERT')
svm = ax.bar(x + width/2, SVM, width, label='SVM')

# Add text for labels, title and custom x-axis tick labels
ax.set_ylabel('Number')
#ax.set_title('Correctly Classified Tweets')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylim([0,1200])
ax.legend()

"""
This function will label the height of the histogram

Input:
    rects: the bar graph to be labeled

Output:
    label for the height of the bar graph
"""
def autolabel(rects):
    # Attach a text label above each bar in *rects*, displaying its height
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(albert)
autolabel(svm)

fig.tight_layout()

plt.show()